In [1]:
from simulator.bb84 import simulation_bb84
from tqdm import tqdm
from numpy import arange

In [3]:
results_list = []
for qber in tqdm(arange(start=0.01, stop=0.16, step = 0.01)):
    simulation_results = simulation_bb84(
        alice_basis_length=10000,
        disturbance_probability=qber,
        cascade_n_passes=4
    )
    results_list.append(simulation_results)
    # How long will it take to perform this simple test?
    # TODO: handling of no error estimation (like in the '93 paper)

  0%|          | 0/15 [31:00<?, ?it/s]


KeyboardInterrupt: 